In [1]:
import os
import pickle

import pandas as pd
import scipy
import datetime
import numpy as np
import tensorflow as tf

from definitions import ROOT_DIR

seed = 2
tf.random.set_seed(seed)
np.random.seed(seed)

import simclr_models
import simclr_utitlities

In [2]:
# Dataset-specific
# working_directory = 'SleepEEG/'
# data_folder = 'SleepEEG'
working_directory = 'MIMIC/'
data_folder = 'MIMIC'
model_name = '20230227-115713_finetuning.hdf5'

In [3]:
# Load preprocessed data
np_test = (np.load(os.path.join(data_folder, 'test_x.npy')),
           np.load(os.path.join(data_folder, 'test_y.npy')))

pretrained_model = tf.keras.models.load_model(os.path.join(working_directory, model_name), compile=False)
probs = pretrained_model.predict(np_test[0])

# print(simclr_utitlities.evaluate_model_simple(pretrained_model.predict(np_test[0]), np_test[1], return_dict=True))

In [4]:
predictions = np.argmax(probs,axis=1)

In [5]:
os.chdir(ROOT_DIR)
subjects = pd.read_csv(os.path.join('datasets', data_folder, 'demographics.csv'))
subjects.head()

,SUBJECT_ID,LANGUAGE,INSURANCE,RELIGION,ETHNICITY,GENDER,AGE
0,22,1,1,1,1,0,1
1,23,1,1,1,1,1,0
2,24,1,1,1,1,1,1
3,25,1,1,1,1,1,1
4,26,1,1,1,0,1,0


In [6]:
test_listfile = pd.read_csv(os.path.join('datasets', data_folder, 'test_listfile.csv'))
test_listfile.head()

,stay,y_true
0,10011_episode1_timeseries.csv,1
1,10026_episode1_timeseries.csv,0
2,10030_episode1_timeseries.csv,0
3,10042_episode1_timeseries.csv,0
4,10094_episode1_timeseries.csv,0


In [7]:
# Get subject from test
test_listfile.loc[:, "subject"] = test_listfile.stay.apply(lambda stay: stay.split("_")[0]).astype(int)
test_listfile.head()

,stay,y_true,subject
0,10011_episode1_timeseries.csv,1,10011
1,10026_episode1_timeseries.csv,0,10026
2,10030_episode1_timeseries.csv,0,10030
3,10042_episode1_timeseries.csv,0,10042
4,10094_episode1_timeseries.csv,0,10094


In [8]:
# merge demographics in test df
test_listfile = test_listfile.merge(subjects, left_on="subject", right_on="SUBJECT_ID", how="left")
test_listfile.head()

,stay,y_true,subject,SUBJECT_ID,LANGUAGE,INSURANCE,RELIGION,ETHNICITY,GENDER,AGE
0,10011_episode1_timeseries.csv,1,10011,10011,1,1,1,0,0,1
1,10026_episode1_timeseries.csv,0,10026,10026,1,1,0,1,0,0
2,10030_episode1_timeseries.csv,0,10030,10030,1,1,1,1,1,0
3,10042_episode1_timeseries.csv,0,10042,10042,1,1,1,1,1,0
4,10094_episode1_timeseries.csv,0,10094,10094,1,1,1,0,1,0


In [9]:
print("Test rows with null demograpphics: {} ({}%)".format(test_listfile.SUBJECT_ID.isna().sum(), test_listfile.SUBJECT_ID.isna().sum()/test_listfile.shape[0]))
test_listfile.drop(columns=['SUBJECT_ID'], inplace=True)
test_listfile.head()

Test rows with null demograpphics: 0 (0.0%)


,stay,y_true,subject,LANGUAGE,INSURANCE,RELIGION,ETHNICITY,GENDER,AGE
0,10011_episode1_timeseries.csv,1,10011,1,1,1,0,0,1
1,10026_episode1_timeseries.csv,0,10026,1,1,0,1,0,0
2,10030_episode1_timeseries.csv,0,10030,1,1,1,1,1,0
3,10042_episode1_timeseries.csv,0,10042,1,1,1,1,1,0
4,10094_episode1_timeseries.csv,0,10094,1,1,1,0,1,0


In [10]:
test_listfile.loc[:, "y_predicted"] = predictions
test_listfile.head()

,stay,y_true,subject,LANGUAGE,INSURANCE,RELIGION,ETHNICITY,GENDER,AGE,y_predicted
0,10011_episode1_timeseries.csv,1,10011,1,1,1,0,0,1,0
1,10026_episode1_timeseries.csv,0,10026,1,1,0,1,0,0,0
2,10030_episode1_timeseries.csv,0,10030,1,1,1,1,1,0,0
3,10042_episode1_timeseries.csv,0,10042,1,1,1,1,1,0,0
4,10094_episode1_timeseries.csv,0,10094,1,1,1,0,1,0,0


In [21]:
# Indicatively testing performance on publicly insured (1 for majority) versus privately insured (0 for minority) subjects
test_listfile_f = test_listfile[test_listfile.INSURANCE == 0]
test_listfile_m = test_listfile[test_listfile.INSURANCE == 1]

In [22]:
print("\n--- Performance on Publicly Insured Subjects ---")
print(simclr_utitlities.evaluate_model_simple(test_listfile_f.y_predicted, test_listfile_f.y_true, is_one_hot=False, return_dict=True))


--- Performance on Publicly Insured Subjects ---
{'Accuracy': 0.8333333333333334, 'AUROC': 0.5, 'AUPRC': 0.16666666666666666, 'Confusion Matrix': array([[30,  0],
       [ 6,  0]], dtype=int64), 'F1 Macro': 0.45454545454545453, 'F1 Micro': 0.8333333333333334, 'F1 Weighted': 0.7575757575757576, 'Precision': 0.4166666666666667, 'Recall': 0.5, 'Kappa': 0.0}


C:\Users\syfan\PycharmProjects\TFC-pretraining\venv\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [23]:
print("\n--- Performance on Privately Insured Subjects ---")
print(simclr_utitlities.evaluate_model_simple(test_listfile_m.y_predicted, test_listfile_m.y_true, is_one_hot=False, return_dict=True))


--- Performance on Privately Insured Subjects ---
{'Accuracy': 0.8846875, 'AUROC': 0.5045520142471137, 'AUPRC': 0.11858091787439613, 'Confusion Matrix': array([[2827,    5],
       [ 364,    4]], dtype=int64), 'F1 Macro': 0.47997750444692344, 'F1 Micro': 0.8846875, 'F1 Weighted': 0.8332206603246354, 'Precision': 0.6651868101257008, 'Recall': 0.5045520142471137, 'Kappa': 0.015816285706665156}
